In [1]:
import os
import pandas as pd
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
from tqdm import tqdm
from pprint import pprint

load_dotenv()

GPT_API_KEY = os.environ["GPT_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
UPSTAGE_API_KEY = os.environ["UPSTAGE_API_KEY"]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [11]:
folder_path = '../data'

In [12]:
doc_loader = CSVLoader(file_path=f"{folder_path}/final_qa_list.csv", encoding='utf-8-sig')
docs_processed = doc_loader.load_and_split(text_splitter=text_splitter)

print(len(docs_processed))
# 309개

309


In [2]:
# Upstage 에서 제공하는 Embedding Model을 활용
upstage_embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [3]:
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-index'

# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=upstage_embedding)

In [4]:
# Prompt Template

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n".join(page.page_content for page in pages)
    return merged

In [6]:
# Query 로 retriever 랑 answer 반환 해주는 함수
def return_llm_result_and_context(query):
    chain = (
        {
            "query": RunnablePassthrough(),
            "context": retriever | merge_pages
        }
        | RunnableParallel(
            {
                "answer": prompt | llm | StrOutputParser(),
                "context": lambda x: x["context"]
            }
        )
    )
    result = chain.invoke(query)
    result_context = result['context']
    result_answer = '.'.join(result['answer'].replace('  ', ' ').split('.'))
    
    return result_context, result_answer

In [114]:
context, result = return_llm_result_and_context('부재중으로 상품이 반송된 경우 배송비는 누가 부담하나요?')

In [116]:
print(f"context: {context}")
print(f"result: {result}")


context: : 34
Question: Q[배송비] 부재중으로 상품이 반송된 경우 배송비는 누가 부담하나요?
Answer: 고객 부재로 반송되거나 연락처를 잘못 기재하여 반송된 경우 상품의 배송비는 고객님께서 부담하셔야 합니다.
keywords: DELIVERY
count: 0: 23
Question: Q[판매자 회수] 상품을 반품할 때 제가 이용하는 택배로 보내도 되나요?
Answer: 만약 반품 상품을 직접 발송하시고 싶다면 원활한 반품 진행을 위해 판매자와 계약된 택배사를 통해 반송해 주시기 바랍니다. 판매자와 계약하지 않은 다른 택배사를 통해 반송할 경우 추가 배송비(추가 반품비용)가 부과될 수 있습니다. 또한 다른 택배사를 이용하여 이미 상품을 반송했다면 마이쿠팡에서 반품 접수를 할 수 없습니다. 반송장 정보를 확인하여 쿠팡 고객센터에서 반품을 접수해 주세요.
 판매자 정보 확인하기 (클릭)
keywords: CANCEL
count: 0: 3
Question: Q[반품비] 반품 비용은 누가 부담하나요?
Answer: 귀책사유에 따라 반품 비용의 부담 주체가 다릅니다. 상품 불량/파손/오배송 등 판매자의 귀책인 경우에는 고객에게 교환/반품 비용이 발생하지 않습니다. 그러나 구매자의 단순 변심인 경우에는 고객에게 반품 비용이 부담될 수 있습니다.
 구매자 단순 변심의 경우 (클릭)
 반품 비용 확인 방법 (클릭)
keywords: CANCEL
count: 0
result: 네, 고객님. 부재중으로 인해 상품이 반송된 경우 배송비는 고객님께서 부담하셔야 합니다. 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다. 확인 후 배송비 관련 안내와 함께 더욱 자세한 내용을 안내해 드리겠습니다.


**조건별 대응 방법:**

* **주문번호 확인 후, 고객 부재로 인한 반송 확인 시:** "고객님, 주문번호 확인 결과 고객님의 부재로 인해 상품이 반송된 것으로 확인되었습니다. 불편을 드려 죄송합니다. 재배송을 원하시면 추가 배송비가 발생하며, 반품을 원

## 질문 생성을 위한 agent step

이슈 발생) 
ChatUpstage 모델의 temperature를 0으로 설정해 놓지 않아서,
매번 Agent를 돌릴 때마다 다른 결과가 나오는 이슈가 발생했었음.

temperature --> Logit 전체를 0에서 무한 사이의 특정 값으로 나눔으로써 확률 분포를 조정
변경 이후 항상 똑같은 평가 값을 반환하는 것을 확인.

In [7]:
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage

# ChatUpstage 객체 생성
llm_client = ChatUpstage(model="solar-mini",  temperature=0)

def call_llm(prompt: str):
    # HumanMessage 객체로 프롬프트 생성
    message = HumanMessage(content=prompt)
    # 모델에 요청 보내기
    response = llm_client.invoke([message])
    # 응답 내용 반환
    return response.content

# 테스트
result = call_llm("This is a test context")
print(result)

I understand that this is a test context. Is there anything specific you would like me to do or assist you with?


### QA 생성 프롬프트

In [8]:
QA_generation_prompt = """
[Context]: {context}\n 
당신의 임무는 주어진 [Context]를 바탕으로 고객이 할 만한 질문을 작성하는 것입니다.
고객은 복잡하게 질문하지 않는 경향이 있습니다.
당신의 사실적 질문은 [Context]에서 구체적인 정보로 답변될 수 있어야 합니다.
사실적 질문은 사용자가 검색 엔진에 물을 수 있는 방식으로 구성되어야 합니다.
이는 사실적 질문이 "해당 글에 따르면"이나 "맥락에 따르면"과 같은 표현을 포함해서는 안 된다는 것을 의미합니다.
또한 사실적 질문은 문서의 Question 내용과 너무 유사하지 않아야 합니다.
 
단, 질문 목록은 무조건 다음 형태로 제공됩니다.: 
Output:::
Q: (사실적 질문)
"""

In [13]:
docs_processed[0].page_content.split('Answer')[0].split(']')[-1].strip()

'상품을 교환/반품하고 싶어요.'

In [102]:
# retriever invoke test
answer = retriever.invoke('Q: 쿠페이 계좌이체에서 계좌 인증 절차는 어떻게 되나요?')

In [104]:
print(answer)

[Document(metadata={'row': 22.0, 'source': '../data/ORDER_main_qna_list.csv'}, page_content=': 22\nQuestion: Q[쿠페이 계좌이체] 계좌 인증은 어떻게 하나요?\nAnswer: 계좌 인증은 다음의 방법으로 진행됩니다. \n 쿠페이 계좌 인증 절차\n① 등록할 은행과 계좌번호 입력\n② 필수 약관 확인 후 동의하기\n③ 본인 계좌로 입금된 1원의 입금자명 확인하여 입력하기\n④ ARS 인증 완료하기\n※ 쿠페이 계좌가 정상적으로 등록되면 안내 알림을 수신할 수 있습니다.\nkeywords: ORDER\ncount: 0'), Document(metadata={'row': 7.0, 'source': '../data/ORDER_main_qna_list.csv'}, page_content=': 7\nQuestion: Q[쿠페이] 쿠팡 쿠페이(Coupay)란 무엇인가요?\nAnswer: 쿠페이(Coupay)란 쿠팡이 제공하는 쉽고 빠르고 안전한 간편 결제 서비스입니다. 아래의 경로를 통해 은행계좌 또는 신용/체크카드 정보를 최초 1회만 등록해 주세요. 번거로운 공인인증서나 보안카드/OTP 기기 대신 고객님께서 설정한 비밀번호를 사용하여 쉽고 빠른 결제가 가능합니다. 또는 쿠페이 머니 전용 충전 계좌를 발급받은 후 금액을 미리 충전(이체) 하여 사용하실 수도 있습니다.\n 쿠페이 등록하기\n마이쿠팡 → 결제수단∙쿠페이 → 결제수단 → 결제수단 등록\nkeywords: ORDER\ncount: 0'), Document(metadata={'row': 31.0, 'source': '../data/ORDER_main_qna_list.csv'}, page_content=': 31\nQuestion: Q[쿠페이] 쿠페이에 가족 명의의 계좌나 휴대폰 등록이 가능한가요?\nAnswer: 아니요. 쿠페이 등록과 계좌이체(충전) 등 결제수단 등록은 본인 명의의 계좌로만 가능합니다.\nkeywords: ORD

### QA 세트 생성

In [18]:
import random
 
N_GENERATIONS = 20  # QA 쌍을 10개만 생성하는 것은 비용 및 시간 고려
 
print(f"Generating {N_GENERATIONS} QA couples...")  
 
outputs = []

# 임의로 N_GENERATIONS만큼 문서 선택
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):  
    # Question 생성
    generate_question = call_llm(QA_generation_prompt.format(context=sampled_context.page_content))
    if generate_question[:2] != "Q:":
        continue
    else:    
        try:
            # pinecone + LLM 유사도 검색 후 답변 생성
            context, answer = return_llm_result_and_context(generate_question)
            outputs.append(
                {
                    "question": generate_question,  # 질문
                    "context": context,  # 문맥
                    "answer": answer,  # 답변
                }
            )
        except Exception as e:
            print(e)
            continue

Generating 20 QA couples...


100%|██████████| 20/20 [01:28<00:00,  4.44s/it]


In [21]:
df = pd.DataFrame(outputs)

df.to_csv("../temp_data/qa_generation.csv", encoding='utf-8-sig', index=False)

# 질문 평가 기준

- Groundedness : 질문은 주어진 컨텍스트로부터 답변할 수 있는 것인가?
- Relevance : 질문이 실제 사용자들과 관련된 것인가? 예를 들어, “transformers 4.29.1이 언제 릴리즈 되었는가?”는 RAG를 사용하는 해당 케이스 관련이 없을 수 있다.
- Stand-alone : 도메인 지식이나 인터넷 접근이 가능한 사람에게 어떤 맥락 없이도 질문을 이해할 수 있는가? 이와 반대되는 예는 특정 블로그 글에서 생성된 질문에 대해 “이 글에서 사용된 함수는 무엇인가?”와 같은 경우이다.

### 기타 에이전트 설정

In [22]:
question_groundedness_critique_prompt = """
주어진 맥락(context)과 질문을 분석해주세요.
당신의 임무는 주어진 맥락으로 해당 질문에 명확하게 답변할 수 있는 정도를 '종합 점수'로 평가하는 것입니다.
점수는 1점부터 5점까지 부여할 수 있으며, 1점은 주어진 맥락으로 질문에 전혀 답변할 수 없음을, 5점은 맥락을 통해 질문에 명확하고 모호하지 않게 답변할 수 있음을 의미합니다.

다음 형식에 맞춰 답변해 주세요:

답변:::
평가: (점수에 대한 근거를 서술해 주세요)
종합 점수: (1부터 5 사이의 숫자로 점수를 매겨주세요)

반드시 '평가:'와 '종합 점수:'를 포함하여 답변해 주세요.

이제 질문과 맥락을 제시하겠습니다.

질문: {question}\n
맥락: {context}\n
답변::: """

In [23]:
question_relevance_critique_prompt = """
질문이 주어집니다.
당신의 임무는 이 질문이 쿠팡에서 상담 업무를 보는 상담사들에게 얼마나 유용한지 대표하는 '종합 점수'를 제공하는 것입니다.
1점은 전혀 유용하지 않음을, 5점은 매우 유용함을 의미하는 1에서 5까지의 점수로 평가하세요.
 
다음과 같이 답변해 주세요:
 
답변:::
평가: (점수를 준 이유에 대한 내용)
종합 점수: (1에서 5 사이의 숫자)
 
'평가:'와 '종합 점수:'에 대한 값이 꼭 포함되어야 합니다.
 
이제 질문이 있습니다.
 
질문: {question}\n
답변::: """
 
question_standalone_critique_prompt = """
질문이 주어집니다.
당신의 임무는 이 질문이 얼마나 맥락에 독립적인지 나타내는 '종합 점수'를 제공하는 것입니다.
1점은 추가 정보가 필요함을 의미하며, 5점은 질문이 스스로 이해가 가능함을 의미합니다.
예를 들어, '맥락에서'나 '문서에서'와 같은 특정 설정을 언급하는 경우 점수는 1이어야 합니다.
질문에 기술적 용어나 약어가 포함되어 있어도 상관없지만, 문서를 통해 질문 내용을 이해할 수 있는 운영자에게 명확해야 합니다.
 
예를 들어, "ViT 모델이 어느 체크포인트에서 가져오는가?"는 특정 맥락이 암시되어 있기 때문에 1점을 받아야 합니다.
 
다음과 같이 답변해 주세요:
 
답변:::
평가: (점수를 준 이유에 대한 내용)
종합 점수: (1에서 5 사이의 숫자)
 
'평가:'와 '종합 점수:'에 대한 값이 꼭 포함되어야 합니다.
 
이제 질문이 있습니다.
 
질문: {question}\n
답변::: """

In [28]:
# 처음 만들었을 때
temp_outputs = outputs.copy()

In [44]:
# 이미 만들어 놨을 때
qa_generate = pd.read_csv('../temp_data/qa_generation.csv', encoding='utf-8-sig')
temp_outputs = []
for output in qa_generate.iterrows():
    question, context, answer = output[1]
    temp_outputs.append({
        'question':question,
        'context':context,
        'answer':answer
    })


In [29]:
temp_outputs

[{'question': 'Q: 새벽/당일배송으로 주문한 상품이 일반배송으로 변경된 이유는 무엇인가요?',
  'context': ': 20\nQuestion: Q[배송] 새벽/당일배송으로 주문했는데 일반배송으로 변경됐어요.\nAnswer: 갑작스러운 주문량 증가 등으로 출고 물류센터에서 새벽/당일 배송이 가능하지 않을 경우 부득이하게 일반배송으로 변경될 수 있는 점 양해 부탁드립니다. \n 배송현황 확인하기\n마이쿠팡 → 주문목록 → 상품선택 → 배송조회\nkeywords: DELIVERY\ncount: 0\n: 18\nQuestion: Q[로켓배송] 새벽배송과 로켓배송을 선택할 수 있나요?\nAnswer: 아니요. 선택할 수 없습니다. 주문하시기 전에 상품 상세페이지에서 배송 시간을 확인해 주세요. 새벽배송 도착 보장 상품에 한하여 새벽배송이 가능합니다. 새벽배송 도착 보장 상품과 다음 날 배송되는 로켓배송 상품을 함께 구매한 경우 상품이 두 번에 나누어져서 각각 배송됩니다.\n 단 로켓와우 상품과 로켓배송 상품을 함께 구매한 경우 새벽배송 상품의 수령 일정을 변경할 수 있습니다. 주문/결제 페이지에서 해당 상품의 새벽도착 여부를 선택해 주세요. 새벽도착을 선택하면 로켓배송 상품과 나누어져 각각 배송되며 새벽도착을 선택하지 않으면 로켓배송 상품과 같은 시간에 함께 배송됩니다. 그리고 이 선택은 다음 주문 시에도 동일하게 적용됩니다. 만약 다음 주문에서 이 선택을 변경하시려면 주문/결제 페이지에서 원하는 옵션으로 다시 선택해 주시기 바랍니다.\nkeywords: DELIVERY\ncount: 0\n: 49\nQuestion: Q[로켓배송] 새벽배송 주문 후 배송지를 변경할 수 있나요?\nAnswer: 상품에 종류에 따라 다릅니다. 새벽배송 전용인 로켓프레시 상품은 주문/결제 완료 후에는 배송지 변경이 불가합니다. 그러나 새벽배송 중 로켓배송 일반 상품은 배송시작 전까지 배송지를 변경할 수 있습니다. 배송지 변경 후에는 아래 경로에서 배송예정일을 다시 확인 부

## Agent Prompt Test

In [47]:
print("Generating critique for each QA couple...")
for output in tqdm(temp_outputs):
    evaluations = {
        "groundedness": call_llm(
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score = int(evaluation.split("종합 점수: ")[-1].strip())
            evaluate = evaluation.split("평가: ")[-1]
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": evaluate,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


100%|██████████| 20/20 [00:42<00:00,  2.13s/it]


In [59]:
from pprint import pprint
pprint(temp_outputs)

[{'answer': '죄송합니다. 새벽/당일 배송으로 주문하신 상품이 일반 배송으로 변경된 점에 대해 불편을 드린 점 사과드립니다. '
            '갑작스러운 주문량 증가 등으로 인해 출고 물류센터에서 새벽/당일 배송이 어려워 부득이하게 일반 배송으로 변경된 것으로 '
            '보입니다. 정확한 상황 확인을 위해 주문하신 주문 번호를 알려주시면 배송 현황을 자세히 확인해 드리고 최대한 빠르게 '
            '해결해 드리겠습니다.\n'
            '\n'
            '\n'
            '**조건별 대응 방법:**\n'
            '\n'
            '* **주문번호 확인 후, 실제로 일반배송으로 변경된 경우:** "고객님의 주문번호를 확인한 결과, 일반 배송으로 '
            '변경된 것을 확인했습니다. 다시 한번 불편을 드린 점 사과드립니다. 현재 배송 예정일은 ~이며, 더 자세한 내용은 '
            '마이쿠팡 > 주문목록 > 상품선택 > 배송조회 에서 확인하실 수 있습니다. 혹시 추가적인 문의사항이 있으시면 언제든지 '
            '연락주세요."\n'
            '\n'
            '* **주문번호 확인 후, 일반배송으로 변경되지 않은 경우 (시스템 오류 등):** "고객님의 주문번호를 확인해 본 '
            '결과, 현재는 새벽/당일 배송으로 예정되어 있습니다. 혹시 배송 정보에 오류가 있으셨다면 죄송합니다. 만약 배송이 '
            '지연되거나 변경되는 경우 바로 연락드리겠습니다. 궁금하신 점이 있으시면 언제든지 문의해주세요."\n'
            '\n'
            '* **주문번호 확인이 어려운 경우:** "정확한 상황 확인을 위해 주문하신 주문 번호 확인 부탁드립니다."\n'
            '\n'
            '\n',
  'context': '

### 품질이 좋지 않은 질문 쌍 필터링

In [60]:
import pandas as pd
import datasets
 
pd.set_option("display.max_colwidth", None)
 
generated_questions = pd.DataFrame.from_dict(temp_outputs)
 
print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 3)
    & (generated_questions["relevance_score"] >= 3)
    & (generated_questions["standalone_score"] >= 3)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
 
eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,Q: 새벽/당일배송으로 주문한 상품이 일반배송으로 변경된 이유는 무엇인가요?,"죄송합니다. 새벽/당일 배송으로 주문하신 상품이 일반 배송으로 변경된 점에 대해 불편을 드린 점 사과드립니다. 갑작스러운 주문량 증가 등으로 인해 출고 물류센터에서 새벽/당일 배송이 어려워 부득이하게 일반 배송으로 변경된 것으로 보입니다. 정확한 상황 확인을 위해 주문하신 주문 번호를 알려주시면 배송 현황을 자세히 확인해 드리고 최대한 빠르게 해결해 드리겠습니다.\n\n\n**조건별 대응 방법:**\n\n* **주문번호 확인 후, 실제로 일반배송으로 변경된 경우:** ""고객님의 주문번호를 확인한 결과, 일반 배송으로 변경된 것을 확인했습니다. 다시 한번 불편을 드린 점 사과드립니다. 현재 배송 예정일은 ~이며, 더 자세한 내용은 마이쿠팡 > 주문목록 > 상품선택 > 배송조회 에서 확인하실 수 있습니다. 혹시 추가적인 문의사항이 있으시면 언제든지 연락주세요.""\n\n* **주문번호 확인 후, 일반배송으로 변경되지 않은 경우 (시스템 오류 등):** ""고객님의 주문번호를 확인해 본 결과, 현재는 새벽/당일 배송으로 예정되어 있습니다. 혹시 배송 정보에 오류가 있으셨다면 죄송합니다. 만약 배송이 지연되거나 변경되는 경우 바로 연락드리겠습니다. 궁금하신 점이 있으시면 언제든지 문의해주세요.""\n\n* **주문번호 확인이 어려운 경우:** ""정확한 상황 확인을 위해 주문하신 주문 번호 확인 부탁드립니다.""\n\n\n",5.0,5.0,5.0
1,Q: 휴대폰 결제 시 인증번호가 오지 않을 때 어떻게 해야 하나요?,"네, 안녕하세요. 휴대폰 결제 인증번호 수신 문제로 불편을 드린 점 진심으로 사과드립니다. 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다. 잠시만 기다려주시면 확인 후 최선을 다해 도와드리겠습니다.\n\n\n**조건별 대응 방법**\n\n1. **단말기 및 통신사 차단 확인:** 고객님의 휴대폰 단말기 또는 통신사의 차단 번호와 스팸 문자함을 먼저 확인해 보겠습니다. 만약 차단되어 있다면 해제 후 다시 인증번호 요청을 시도해 보시기 바랍니다.\n\n2. **스팸 문자함 확인:** 스팸 문자함에 인증번호가 잘못 분류되었을 가능성이 있습니다. 스팸 문자함을 확인하여 인증번호가 있는지 확인해 주세요.\n\n3. **통신사 고객센터 문의:** 위 방법으로도 해결되지 않을 경우, 이용 중인 휴대폰 통신사 고객센터에 문의하여 인증번호 수신 문제에 대해 문의하시는 것이 좋습니다. 통신사 측에서 문제 해결을 도와드릴 수 있습니다. 필요시 통신사 고객센터 연결을 도와드리겠습니다.\n\n4. **주문번호 확인 후 추가 확인:** 주문번호 확인 후, 고객님의 휴대폰 번호와 명의가 정확하게 입력되었는지, 혹시 요금 미납이나 다른 문제는 없는지 추가적으로 확인해 드리겠습니다.\n",5.0,5.0,5.0
2,Q: 무통장입금으로 주문한 후 다른 은행으로 변경할 수 있나요?,"네, 안녕하세요. 무통장입금으로 주문하신 후 은행 변경에 대해 문의하셨네요. 죄송하지만, 주문 완료 후에는 다른 은행으로 변경이 불가능합니다. 주문 취소 후 다시 원하시는 은행으로 주문해주시면 됩니다.\n\n\n정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.\n\n\n**조건별 대응 방법:**\n\n* **주문번호 확인 후 주문 취소 가능한 경우:** ""주문번호 확인했습니다. 주문 취소를 도와드리겠습니다. 취소 완료 후 다시 원하시는 은행으로 주문해주시면 됩니다. 불편을 드려 죄송합니다.""\n\n* **주문번호 확인 후 주문 취소가 불가능한 경우 (예: 이미 입금 완료, 배송 시작 등):** ""주문번호 확인했습니다. 현재 주문 상태로는 은행 변경이 어렵습니다. 상황에 따라 취소 및 재주문이 가능할 수 있으니, 자세한 내용은 [고객센터 번호]로 문의해주시면 친절하게 안내해 드리겠습니다. 불편을 드려 죄송합니다.""\n\n* **주문번호 확인 불가능한 경우:** ""정확한 상담을 위해 주문번호 확인이 필요합니다. 주문번호를 찾아 다시 문의해주시면 감사하겠습니다.""\n",5.0,5.0,5.0
3,Q: 로켓직구에서 건강기능식품은 몇 병까지 구매할 수 있나요?,"네, 안녕하세요. 로켓직구 건강기능식품 구매 관련 문의 주셨네요. 건강기능식품은 최대 6병까지 구매 가능하며, 6병을 초과하는 경우 면세 범위를 초과하여 세관에서 폐기될 수 있다는 점 참고 부탁드립니다. 더 궁금하신 점이나 추가적인 도움이 필요하시면 언제든지 다시 문의주세요.\n\n\n**조건별 대응 방법:**\n\n해당 질문에 대한 답변은 context에 명시되어 있으며, 추가적인 정보는 필요하지 않습니다. 따라서 별도의 조건별 대응 방법은 없습니다.\n",1.0,5.0,5.0
4,Q: 쿠팡에서 무통장입금을 시도할 때 거래불가로 입금할 수 없는 경우 어떻게 해야 하나요?,"네, 안녕하세요. 쿠팡 무통장 입금 시 거래 불가로 입금이 안 되셔서 불편을 드린 점 진심으로 사과드립니다. 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다. 잠시만 기다려주시면 확인 후 빠르게 해결해 드리겠습니다.\n\n\n**각 조건별 대응 방법**\n\n1. **가상계좌 확인:** 고객님께서 입력하신 가상계좌 번호가 주문 시 발급받은 가상계좌 번호와 일치하는지 확인합니다. 일치하지 않을 경우, 정확한 가상계좌 번호를 안내하고 재입금을 안내합니다.\n\n2. **입금 금액 확인:** 주문 금액과 입금 금액이 일치하는지 확인합니다. 금액이 다를 경우, 차액을 안내하고 정확한 금액을 입금하도록 안내합니다. 만약 여러 상품을 한꺼번에 입금하려고 하셨다면, 각 주문별 가상계좌로 입금해야 함을 설명하고, 마이쿠팡 > 주문목록 > 주문 상세보기에서 각 주문의 가상계좌를 확인하는 방법을 안내합니다.\n\n3. **입금 기한 확인:** 주문 완료 후 24시간 이내 입금해야 함을 안내하고, 입금 기한이 지났다면 주문 취소 후 재주문을 안내합니다. 판매 종료 시간이 24시간 이내인 상품이 포함된 주문의 경우, 해당 상품의 판매 종료 시간을 기준으로 입금 기한을 설명합니다.\n\n4. **ATM 입금 확인:** ATM기기에서는 100원 단위 입금이 불가능하다는 점을 알려드리고, 은행 창구 또는 계좌이체를 이용하도록 안내합니다.\n\n5. **은행 확인:** 일부 은행에서 가상계좌 입금이 불가능할 수 있다는 점을 알려드리고, 은행 창구 또는 계좌이체를 이용하도록 안내합니다. 필요하다면 다른 은행을 이용해 입금하는 방법을 제안합니다.\n\n6. **해외 계좌 확인:** 만약 해외 계좌에서 입금을 시도하셨다면, 해외 계좌는 지원되지 않는다는 점을 알려드리고 국내 계좌를 이용하도록 안내합니다.\n",5.0,5.0,5.0
5,Q: 쿠페이에 등록한 카드를 재발급 받으면 다시 등록해야 하나요?,"네, 안녕하세요. 쿠페이 담당 상담사입니다. 쿠페이에 등록하신 카드를 재발급 받으셨다면 카드번호가 변경되었기 때문에 재등록이 필요합니다. 쿠페이 앱 또는 웹사이트에 접속하셔서 기존 카드를 삭제하신 후 새로 발급받으신 카드를 등록해주시면 됩니다. 자세한 방법은 쿠페이 앱 내 '결제수단 관리' 메뉴에서 확인하실 수 있습니다. 혹시 어려움이 있으시다면 언제든지 다시 연락주세요. 최선을 다해 도와드리겠습니다.\n\n\n**조건별 대응 방법:**\n\n(본 질문의 경우, context 내 정보만으로 충분히 답변 가능하므로 별도의 조건별 대응 방법은 없습니다.)\n",5.0,5.0,3.0
6,Q: 신규 주문 시 이전에 주문한 상품과 한 번에 받고 싶지 않은 경우 어떻게 해야 하나요?,"안녕하세요, 고객님. 신규 주문 시 이전 주문과 함께 받고 싶지 않으시다니 불편을 드려 죄송합니다. 주문/결제 단계에서 '따로 받기'를 선택하시면 됩니다. 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.\n\n\n**조건별 대응 방법:**\n\n* **'한 번에 받기' 옵션이 표시되는 경우:** 고객님께서 '한 번에 받기' 옵션을 선택하지 않고 '따로 받기'를 선택하시

Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,Q: 새벽/당일배송으로 주문한 상품이 일반배송으로 변경된 이유는 무엇인가요?,"죄송합니다. 새벽/당일 배송으로 주문하신 상품이 일반 배송으로 변경된 점에 대해 불편을 드린 점 사과드립니다. 갑작스러운 주문량 증가 등으로 인해 출고 물류센터에서 새벽/당일 배송이 어려워 부득이하게 일반 배송으로 변경된 것으로 보입니다. 정확한 상황 확인을 위해 주문하신 주문 번호를 알려주시면 배송 현황을 자세히 확인해 드리고 최대한 빠르게 해결해 드리겠습니다.\n\n\n**조건별 대응 방법:**\n\n* **주문번호 확인 후, 실제로 일반배송으로 변경된 경우:** ""고객님의 주문번호를 확인한 결과, 일반 배송으로 변경된 것을 확인했습니다. 다시 한번 불편을 드린 점 사과드립니다. 현재 배송 예정일은 ~이며, 더 자세한 내용은 마이쿠팡 > 주문목록 > 상품선택 > 배송조회 에서 확인하실 수 있습니다. 혹시 추가적인 문의사항이 있으시면 언제든지 연락주세요.""\n\n* **주문번호 확인 후, 일반배송으로 변경되지 않은 경우 (시스템 오류 등):** ""고객님의 주문번호를 확인해 본 결과, 현재는 새벽/당일 배송으로 예정되어 있습니다. 혹시 배송 정보에 오류가 있으셨다면 죄송합니다. 만약 배송이 지연되거나 변경되는 경우 바로 연락드리겠습니다. 궁금하신 점이 있으시면 언제든지 문의해주세요.""\n\n* **주문번호 확인이 어려운 경우:** ""정확한 상황 확인을 위해 주문하신 주문 번호 확인 부탁드립니다.""\n\n\n",5.0,5.0,5.0
1,Q: 휴대폰 결제 시 인증번호가 오지 않을 때 어떻게 해야 하나요?,"네, 안녕하세요. 휴대폰 결제 인증번호 수신 문제로 불편을 드린 점 진심으로 사과드립니다. 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다. 잠시만 기다려주시면 확인 후 최선을 다해 도와드리겠습니다.\n\n\n**조건별 대응 방법**\n\n1. **단말기 및 통신사 차단 확인:** 고객님의 휴대폰 단말기 또는 통신사의 차단 번호와 스팸 문자함을 먼저 확인해 보겠습니다. 만약 차단되어 있다면 해제 후 다시 인증번호 요청을 시도해 보시기 바랍니다.\n\n2. **스팸 문자함 확인:** 스팸 문자함에 인증번호가 잘못 분류되었을 가능성이 있습니다. 스팸 문자함을 확인하여 인증번호가 있는지 확인해 주세요.\n\n3. **통신사 고객센터 문의:** 위 방법으로도 해결되지 않을 경우, 이용 중인 휴대폰 통신사 고객센터에 문의하여 인증번호 수신 문제에 대해 문의하시는 것이 좋습니다. 통신사 측에서 문제 해결을 도와드릴 수 있습니다. 필요시 통신사 고객센터 연결을 도와드리겠습니다.\n\n4. **주문번호 확인 후 추가 확인:** 주문번호 확인 후, 고객님의 휴대폰 번호와 명의가 정확하게 입력되었는지, 혹시 요금 미납이나 다른 문제는 없는지 추가적으로 확인해 드리겠습니다.\n",5.0,5.0,5.0
2,Q: 무통장입금으로 주문한 후 다른 은행으로 변경할 수 있나요?,"네, 안녕하세요. 무통장입금으로 주문하신 후 은행 변경에 대해 문의하셨네요. 죄송하지만, 주문 완료 후에는 다른 은행으로 변경이 불가능합니다. 주문 취소 후 다시 원하시는 은행으로 주문해주시면 됩니다.\n\n\n정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.\n\n\n**조건별 대응 방법:**\n\n* **주문번호 확인 후 주문 취소 가능한 경우:** ""주문번호 확인했습니다. 주문 취소를 도와드리겠습니다. 취소 완료 후 다시 원하시는 은행으로 주문해주시면 됩니다. 불편을 드려 죄송합니다.""\n\n* **주문번호 확인 후 주문 취소가 불가능한 경우 (예: 이미 입금 완료, 배송 시작 등):** ""주문번호 확인했습니다. 현재 주문 상태로는 은행 변경이 어렵습니다. 상황에 따라 취소 및 재주문이 가능할 수 있으니, 자세한 내용은 [고객센터 번호]로 문의해주시면 친절하게 안내해 드리겠습니다. 불편을 드려 죄송합니다.""\n\n* **주문번호 확인 불가능한 경우:** ""정확한 상담을 위해 주문번호 확인이 필요합니다. 주문번호를 찾아 다시 문의해주시면 감사하겠습니다.""\n",5.0,5.0,5.0
4,Q: 쿠팡에서 무통장입금을 시도할 때 거래불가로 입금할 수 없는 경우 어떻게 해야 하나요?,"네, 안녕하세요. 쿠팡 무통장 입금 시 거래 불가로 입금이 안 되셔서 불편을 드린 점 진심으로 사과드립니다. 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다. 잠시만 기다려주시면 확인 후 빠르게 해결해 드리겠습니다.\n\n\n**각 조건별 대응 방법**\n\n1. **가상계좌 확인:** 고객님께서 입력하신 가상계좌 번호가 주문 시 발급받은 가상계좌 번호와 일치하는지 확인합니다. 일치하지 않을 경우, 정확한 가상계좌 번호를 안내하고 재입금을 안내합니다.\n\n2. **입금 금액 확인:** 주문 금액과 입금 금액이 일치하는지 확인합니다. 금액이 다를 경우, 차액을 안내하고 정확한 금액을 입금하도록 안내합니다. 만약 여러 상품을 한꺼번에 입금하려고 하셨다면, 각 주문별 가상계좌로 입금해야 함을 설명하고, 마이쿠팡 > 주문목록 > 주문 상세보기에서 각 주문의 가상계좌를 확인하는 방법을 안내합니다.\n\n3. **입금 기한 확인:** 주문 완료 후 24시간 이내 입금해야 함을 안내하고, 입금 기한이 지났다면 주문 취소 후 재주문을 안내합니다. 판매 종료 시간이 24시간 이내인 상품이 포함된 주문의 경우, 해당 상품의 판매 종료 시간을 기준으로 입금 기한을 설명합니다.\n\n4. **ATM 입금 확인:** ATM기기에서는 100원 단위 입금이 불가능하다는 점을 알려드리고, 은행 창구 또는 계좌이체를 이용하도록 안내합니다.\n\n5. **은행 확인:** 일부 은행에서 가상계좌 입금이 불가능할 수 있다는 점을 알려드리고, 은행 창구 또는 계좌이체를 이용하도록 안내합니다. 필요하다면 다른 은행을 이용해 입금하는 방법을 제안합니다.\n\n6. **해외 계좌 확인:** 만약 해외 계좌에서 입금을 시도하셨다면, 해외 계좌는 지원되지 않는다는 점을 알려드리고 국내 계좌를 이용하도록 안내합니다.\n",5.0,5.0,5.0
5,Q: 쿠페이에 등록한 카드를 재발급 받으면 다시 등록해야 하나요?,"네, 안녕하세요. 쿠페이 담당 상담사입니다. 쿠페이에 등록하신 카드를 재발급 받으셨다면 카드번호가 변경되었기 때문에 재등록이 필요합니다. 쿠페이 앱 또는 웹사이트에 접속하셔서 기존 카드를 삭제하신 후 새로 발급받으신 카드를 등록해주시면 됩니다. 자세한 방법은 쿠페이 앱 내 '결제수단 관리' 메뉴에서 확인하실 수 있습니다. 혹시 어려움이 있으시다면 언제든지 다시 연락주세요. 최선을 다해 도와드리겠습니다.\n\n\n**조건별 대응 방법:**\n\n(본 질문의 경우, context 내 정보만으로 충분히 답변 가능하므로 별도의 조건별 대응 방법은 없습니다.)\n",5.0,5.0,3.0
7,Q: 로켓프레시 상품을 구매하면 새벽배송이 가능한가요?,"네, 로켓프레시 상품은 새벽배송이 가능합니다. 최대한 빠르고 신선하게 오전 7시 전에 배송되도록 노력하고 있지만, 주문량이나 날씨 등의 상황에 따라 배송 시간이 변경될 수 있는 점 양해 부탁드립니다. 정확한 배송 시간은 주문 완료 후 확인하실 수 있습니다.\n\n\n**조건별 대응 방법:**\n\n* **일반적인 경우:** 위 답변 그대로 안내한다.\n* **배송 지연 또는 배송 불가능한 경우:** ""불편을 드려 죄송합니다. 정확한 상황 확인을 위해 주문하신 주문 번호 확인 부탁드립니다. 확인 후 다시 연락드리겠습니다.""\n* **로켓와우 미가입 또는 서비스 지역 외인 경우:** ""죄송합니다만, 고객님의 현재 배송지는 로켓와우(새벽배송 포함) 서비스 지역이 아닐 수 있습니다. 마이쿠팡 > MY정보 > 배송지 관리에서 로켓와우/로켓프레시 가능 여부를 확인해주세요. 서비스 지역 확장을 위해 노력하고 있습니다.""\n* **로켓프레시 상품과 로켓배송 상품을 함께 주문한 경우:** ""로켓프레시 상품과 로켓배송 상품을 함께 주문하셨다면, 상품이 따로 배송될 수

In [61]:
eval_dataset.to_csv("../temp_data/eval_qa_generation.csv", encoding='utf-8-sig', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

55573

In [65]:
def merge_pages(pages):
    merged = "\n".join(page.page_content for page in pages)
    return merged

template = """
[context]: {context}
---
[질의]: {query}

7년 이상의 경력을 가진 상담사라고 생각하고, 위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답해주세요.
최대한 문장을 쉼표로 끊어서 대답하기 보다는 온점으로 문장을 끊어주세요. 
문장의 마무리는 '~요' 보다는 '~다'로 끝나는 쪽이 전문적으로 보입니다.

또한, 상담사는 가능한 선에서 직접 확인+안내+해결을 도와주는 직원이므로 직접 확인 후 해결까지 돕는 방향으로 작성해 주세요.
그리고, 사용자의 편의를 위해 서비스 특성 상 쿠션어를 사용하시면 좋습니다.
쿠션어의 예시는 다음과 같습니다.
예시)
불편을 드려 죄송합니다.
번거로우시겠지만~
~하는 점 양해 부탁드립니다.
~할 예정입니다.
~를 부탁드립니다.

위 사항들을 종합해서 2~3줄로 상담사가 활용하기 좋게 대본을 만들어 주세요.

만약, 조건별로 안내 내용이 다른 경우
1차 응대 (양해멘트 or 1차 안내 등) + 정보 확인 멘트로 대본을 구성하면 됩니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다." 입니다.
문서의 아래에 각 조건별 대응 방법을 기술해 주세요.

단, 제일 중요한 것은 [context] 정보에 없는 내용을 답해서는 안됩니다. [context]에 정보가 없거나 문서들의 유사성이 0.2 이하로 떨어질 경우, "문의주신 내용은 확인이 필요하여 지금 답변드리기 어려울 것 같습니다. 번거로우시겠지만 확인 후에 다시 연락드려도 괜찮을까요?" 라고 답해주세요.
"""

chain = (
        {"query": RunnablePassthrough(), "context": retriever | merge_pages}
        | prompt
        | llm
        | StrOutputParser()
)

ans = chain.invoke('현금영수증 또는 신용카드 매출전표를 받는 방법은 무엇인가요?')
print(ans)

안녕하세요, 고객님. 현금영수증 또는 신용카드 매출전표 발급에 대해 문의하셨네요.  상품 구매 후에는 영수증/매출전표가 자동 발급되지만, PC와 모바일에서 확인 방법이 조금 다르니 자세히 안내해 드리겠습니다.  정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.


**조건별 대응 방법**

* **PC 이용 고객:**  마이쿠팡 > MY쇼핑 > 영수증 조회/출력에서 기간별 일괄 확인 및 출력이 가능합니다.  각 주문별 확인도 가능하니 편하신 방법으로 이용하시면 됩니다.

* **모바일 이용 고객:** 마이쿠팡 > MY쇼핑 > 주문목록에서 해당 주문의 상세보기를 통해 영수증을 확인하고 출력할 수 있습니다.  각 주문별로만 확인 가능한 점 양해 부탁드립니다.

* **2015년 7월 16일 이전 무통장 입금 고객:** 국세청 홈페이지에서 확인하셔야 합니다.  불편을 드려 죄송합니다.

* **휴대폰 결제 고객:** 결제영수증 조회/발급 메뉴를 통해 결제대행사(PG사)의 거래내역을 확인하시거나, 다음 달 휴대폰 요금 납부 후 통신사에서 발행되는 영수증을 확인하시면 됩니다.

* **직구 상품 고객:** 해외 판매자의 사업장 소재지 법률 적용으로 현금영수증 발급이 불가능합니다.  이 점 양해 부탁드립니다.



